In [91]:
import pysal.esda.mapclassify as mapclassify
import pandas as pd
import geopandas as gpd
from mapboxgl.utils import *
from mapboxgl.viz import *
from sklearn.cluster import KMeans
import json
from shapely.geometry import Polygon, Point

In [92]:
df = pd.read_csv('data/voters.csv')
df.columns = ['Name', 'Address', 'lat', 'lon', 'zipcode']
pdf = gpd.read_file('data/precincts_sdcc_d6.json')
vdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))

In [110]:
volunteers = 7
precinct = '130960'

def create_clusters(dataframe, volunteers):
    kmeans = KMeans(n_clusters=volunteers).fit(dataframe)
    return kmeans

def get_mappings(kmeans):
    return kmeans.labels_

pvdf = vdf[pdf.loc[pdf['code'] == precinct]['geometry'].apply(lambda x: vdf['geometry'].within(x)).any()]
kmeans = create_clusters(pvdf[['lat','lon']], volunteers)
pvdf['cluster'] = get_mappings(kmeans)
pvdf

/Users/mayankkulkarni/Documents/CSCI_577A/Field_Progress_Map/venv/lib/python2.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,Name,Address,lat,lon,zipcode,geometry,cluster
6865,Lavinnia Sweigart,8990 BRENTFORD AVE,32.920173,-117.131861,130960,POINT (-117.13186 32.92017),3
6866,Mercedese Irmgard Thero,8880 BRENTFORD AVE,32.920137,-117.133607,130960,POINT (-117.13361 32.92014),6
6867,Denkevitz Rener,8950 BRENTFORD AVE,32.920159,-117.132555,130960,POINT (-117.13255 32.92016),1
6868,Assent Gantvoort,8969 LIBRA DR,32.920452,-117.132507,130960,POINT (-117.13251 32.92045),1
6869,Keuna Malmgren,8953 LIBRA DR,32.920445,-117.132834,130960,POINT (-117.13283 32.92045),1
6870,Jimyah Wokwicz,8776 BRENTFORD AVE,32.920105,-117.135170,130960,POINT (-117.13517 32.92011),0
6871,Cannette Mahalick,8921 LIBRA DR,32.920426,-117.133653,130960,POINT (-117.13365 32.92043),6
6872,Liliya Rallison,8850 BRENTFORD AVE,32.920126,-117.134128,130960,POINT (-117.13413 32.92013),2
6873,Fugate Standerwick,8961 LIBRA DR,32.920449,-117.132670,130960,POINT (-117.13267 32.92045),1
6874,Yeicob Domingues,8880 BRENTFORD AVE,32.920137,-117.133607,130960,POINT (-117.13361 32.92014),6


In [108]:
mapbox_token = 'pk.eyJ1IjoidWJhY2hyaXMiLCJhIjoiY2sxYjczdWhpMGZuMzNjb2I5OGlqb3gwaCJ9.iLrtxaVXfhsJM0iyWwdQ5Q'

In [113]:
df_to_geojson(pvdf, filename='points1.geojson', 
                    properties = ['Name','Address','zipcode','cluster'],
                     lat='lat', lon='lon', precision=10)

{'feature_count': 16, 'filename': 'points1.geojson', 'type': 'file'}

In [114]:
color_breaks = mapclassify.Natural_Breaks(pvdf['cluster'], k=volunteers, initial=0).bins
color_stops = create_color_stops(color_breaks, colors='YlGnBu')

# Create the viz from the dataframe
viz = CircleViz('points1.geojson',
                access_token=mapbox_token, 
                height='1000px',
                color_property = "",
                color_stops = color_stops,
                center = (-117.176, 32.8560),
                zoom = 10,
                below_layer = 'waterway-label')

viz.show()

In [115]:
viz = CircleViz('points1.geojson',
                access_token=mapbox_token, 
                height='1000px',
                color_property = "cluster",
                color_stops = color_stops,
                center = (-117.176, 32.8560),
                zoom = 11,
                below_layer = 'waterway-label')

viz.show()